In [1]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio.features

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer as pc
import xrspatial.multispectral as ms
import odc
from odc.stac import stac_load
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()
# Pass your API key here
pc.settings.set_subscription_key('9407a5ebfcdf47809130408b746d7c73')
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd

In [2]:
crop_presence_data = pd.read_csv("Crop_Location_Data_20221201.csv")

In [3]:
# Define the pixel resolution for the final product
# Define the scale according to our selected crs, so we will use degrees

resolution = 10  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for crs=4326 
box_size_deg = 0.0004 # Surrounding box in degrees, yields approximately 5x5 pixel region
allrvis = []

In [4]:
# Rice Crop Field in An Giang, Vietnam
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    lat_long = coordinates # Lat-Lon centroid location
    
    lat_long=lat_long.replace('(','').replace(')','').replace(' ','').split(',')
    
    min_lon = float(lat_long[1])-box_size_deg/2
    min_lat = float(lat_long[0])-box_size_deg/2
    max_lon = float(lat_long[1])+box_size_deg/2
    max_lat = float(lat_long[0])+box_size_deg/2

    bbox = (min_lon, min_lat, max_lon, max_lat)
    time_of_interest = "2021-12-01/2022-08-30"
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_of_interest)
    items = list(search.get_all_items()) 
    data = stac_load(items,bands=["vv", "vh"], patch_url=pc.sign, bbox=bbox, crs="EPSG:4326", resolution=scale)
    mean = data.mean(dim=['latitude','longitude']).compute()
    dop = (mean.vv / (mean.vv + mean.vh))
    m = 1 - dop
    rvi = (np.sqrt(dop))*((4*mean.vh)/(mean.vv + mean.vh))
    allrvis.append(rvi)

100%|██████████| 600/600 [1:39:03<00:00,  9.91s/it]


In [5]:
# Convert the list of RVI data to a list of DataFrames
rvi_dfs = [rvi.to_dataframe(name="RVI").reset_index() for rvi in allrvis]

# Concatenate the list of DataFrames
rvi_df = pd.concat(rvi_dfs, axis=0, ignore_index=True)

# Save the DataFrame to a CSV file
rvi_df.to_csv("allrvis.csv", index=False)

In [6]:
# Create a new DataFrame with the filtered RVI list, the original coordinates, and the Class of Land column
rvi_df = pd.DataFrame({
    'Latitude and Longitude': crop_presence_data['Latitude and Longitude'], 
    'Class of Land': crop_presence_data['Class of Land'],
    'RVI': allrvis
})

# Save the DataFrame to a CSV file
rvi_df.to_csv("allrvis_with_coordinates_and_class.csv", index=False)


In [7]:
def extract_features(rvi_df):
    return {
        'mean_rvi': rvi_df.mean(),
        'min_rvi': rvi_df.min(),
        'max_rvi': rvi_df.max(),
        'std_rvi': rvi_df.std()
    }

# Apply the feature extraction function to the RVI_Time_Series column
features = rvi_df['RVI'].apply(extract_features)

# Convert the extracted features to a DataFrame
features_df = pd.DataFrame(list(features))

In [34]:
# Calculate the mean of the data across the sample region
final_df = pd.concat([rvi_df, features_df], axis=1)

In [62]:
# Calculate RVI
from sklearn.model_selection import train_test_split

# Define input features (X) and target variable (y)
X = final_df[['min_rvi', 'max_rvi', 'std_rvi']] #'mean_rvi', 
y = final_df['Class of Land']

# Split the dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)

In [63]:


# Create a RandomForest classifier and fit it to the training data
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Accuracy: 0.9666666666666667
              precision    recall  f1-score   support

    Non Rice       0.97      0.96      0.97       144
        Rice       0.96      0.97      0.97       156

    accuracy                           0.97       300
   macro avg       0.97      0.97      0.97       300
weighted avg       0.97      0.97      0.97       300



In [64]:
y_pred = clf.predict(final_df[[ 'min_rvi', 'max_rvi', 'std_rvi']]) #'mean_rvi',

accuracy = accuracy_score(final_df['Class of Land'], y_pred)
print("Accuracy:", accuracy)
print(classification_report(final_df['Class of Land'], y_pred))

final_df['Predicted Class'] = y_pred

Accuracy: 0.9833333333333333
              precision    recall  f1-score   support

    Non Rice       0.99      0.98      0.98       300
        Rice       0.98      0.99      0.98       300

    accuracy                           0.98       600
   macro avg       0.98      0.98      0.98       600
weighted avg       0.98      0.98      0.98       600



In [38]:
final_df.to_csv("finalpredict.csv", index=False)

In [39]:
crop_presence_data = pd.read_csv("challenge_1_submission_template.csv")

In [40]:
# Rice Crop Field in An Giang, Vietnam
allrvischallenge = []
for coordinates in tqdm(crop_presence_data['id']):
    lat_long = coordinates # Lat-Lon centroid location
    
    lat_long=lat_long.replace('(','').replace(')','').replace(' ','').split(',')
    
    min_lon = float(lat_long[1])-box_size_deg/2
    min_lat = float(lat_long[0])-box_size_deg/2
    max_lon = float(lat_long[1])+box_size_deg/2
    max_lat = float(lat_long[0])+box_size_deg/2

    bbox = (min_lon, min_lat, max_lon, max_lat)
    time_of_interest = "2021-12-01/2022-08-30"
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_of_interest)
    items = list(search.get_all_items()) 
    data = stac_load(items,bands=["vv", "vh"], patch_url=pc.sign, bbox=bbox, crs="EPSG:4326", resolution=scale)
    mean = data.mean(dim=['latitude','longitude']).compute()
    dop = (mean.vv / (mean.vv + mean.vh))
    m = 1 - dop
    rvi = (np.sqrt(dop))*((4*mean.vh)/(mean.vv + mean.vh))
    allrvischallenge.append(rvi)

100%|██████████| 250/250 [41:39<00:00, 10.00s/it]


In [41]:
# Convert the list of RVI data to a list of DataFrames
rvi_dfschallenge = [rvi.to_dataframe(name="RVI").reset_index() for rvi in allrvischallenge]

# Concatenate the list of DataFrames
rvi_dfchallenge = pd.concat(rvi_dfschallenge, axis=0, ignore_index=True)

# Save the DataFrame to a CSV file
rvi_dfchallenge.to_csv("allrvischallenge.csv", index=False)

In [42]:
# Create a new DataFrame with the filtered RVI list, the original coordinates, and the Class of Land column
rvi_dfchallenge = pd.DataFrame({
    'Latitude and Longitude': crop_presence_data['id'],
    'RVI': allrvischallenge
})

# Save the DataFrame to a CSV file
rvi_dfchallenge.to_csv("allrvis_with_coordinates_and_class.csv", index=False)

In [43]:
def extract_features(rvi_df):
    return {
        'mean_rvi': rvi_df.mean(),
        'min_rvi': rvi_df.min(),
        'max_rvi': rvi_df.max(),
        'std_rvi': rvi_df.std()
    }

# Apply the feature extraction function to the RVI_Time_Series column
featureschallenge = rvi_dfchallenge['RVI'].apply(extract_features)

# Convert the extracted features to a DataFrame
features_dfchallenge = pd.DataFrame(list(featureschallenge))

In [44]:
# Calculate the mean of the data across the sample region
final_dfchallenge = pd.concat([rvi_dfchallenge, features_dfchallenge], axis=1)

In [45]:
y_predchallenge = clf.predict(final_dfchallenge[['min_rvi', 'max_rvi', 'std_rvi']])#'mean_rvi', 



crop_presence_data['target'] = y_predchallenge

crop_presence_data.to_csv("challenge1.csv", index=False)